In [1]:
import torch
import numpy as np

def optimize_markowitz(
    covariance,
    expected_return,
    risk_coefficient=1,
    number_of_bits=1,
    agents=128,
    max_steps=1000,
    time_step=0.05,
    verbose=True
):
    """
    Optimize a Markowitz portfolio using the Discrete Simulated Bifurcation algorithm.
    
    Parameters:
    -----------
    covariance : torch.Tensor or numpy.ndarray
        Covariance matrix of asset returns
    expected_return : torch.Tensor or numpy.ndarray
        Expected returns vector
    risk_coefficient : float
        Risk aversion coefficient (higher means more risk-averse)
    number_of_bits : int
        Number of bits to represent each asset quantity
    agents : int
        Number of parallel optimization agents
    max_steps : int
        Maximum number of simulation steps
    time_step : float
        Size of each time step in the simulation
    verbose : bool
        Whether to print progress information
    
    Returns:
    --------
    portfolio : torch.Tensor
        Optimized portfolio weights
    gain : float
        Expected gain of the optimized portfolio
    """
    # Convert inputs to tensors if needed
    if not isinstance(covariance, torch.Tensor):
        covariance = torch.tensor(covariance, dtype=torch.float32)
    if not isinstance(expected_return, torch.Tensor):
        expected_return = torch.tensor(expected_return, dtype=torch.float32)
    
    # Get dimensions
    assets = covariance.shape[0]
    
    # Prepare optimization coefficients (Markowitz to Ising model)
    matrix = -0.5 * risk_coefficient * covariance
    vector = expected_return
    
    # Initialize position and momentum randomly
    position = 2 * torch.rand(assets, agents) - 1
    momentum = 2 * torch.rand(assets, agents) - 1
    
    # Calculate scaling parameter for stability
    scaling = 0.5 * (assets - 1)**0.5 / torch.sqrt(torch.sum(matrix**2))
    
    # Initialize pressure parameter
    pressure_slope = 0.01
    
    # Run symplectic integration
    if verbose:
        print("Starting Discrete Simulated Bifurcation optimization...")
    
    for step in range(max_steps):
        # Calculate current pressure (annealing parameter)
        pressure = min(time_step * step * pressure_slope, 1.0)
        
        # Calculate coefficients
        momentum_coef = time_step * (pressure - 1.0)
        position_coef = time_step
        quadratic_coef = time_step * scaling
        
        # Perform integration step (discrete SB)
        # 1. Update momentum
        momentum += momentum_coef * position
        
        # 2. Update position
        position += position_coef * momentum
        
        # 3. Update momentum using quadratic term with sign function
        momentum = momentum + quadratic_coef * (matrix @ torch.sign(position))
        
        # 4. Enforce boundary conditions
        momentum[torch.abs(position) > 1.0] = 0.0
        position = torch.clamp(position, -1.0, 1.0)
        
        # Optional progress report
        if verbose and step % 100 == 0:
            print(f"Step {step}/{max_steps}")
    
    # Sample final spins (convert positions to spins)
    spins = torch.where(position >= 0.0, 1.0, -1.0)
    
    # Convert spins to integer portfolio
    max_val = 2**number_of_bits - 1
    portfolios = ((spins + 1) / 2) * max_val
    
    # Evaluate all solutions
    gains = torch.zeros(agents)
    for i in range(agents):
        weights = portfolios[:, i]
        expected_gain = weights @ expected_return
        risk = 0.5 * risk_coefficient * weights @ covariance @ weights
        gains[i] = expected_gain - risk
    
    # Find best solution
    best_idx = torch.argmax(gains)
    best_portfolio = portfolios[:, best_idx]
    best_gain = gains[best_idx].item()
    
    if verbose:
        print(f"\nOptimization complete!")
        print(f"Best portfolio: {best_portfolio}")
        print(f"Expected gain: {best_gain:.4f}")
    
    return best_portfolio, best_gain


# Example usage
if __name__ == "__main__":
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Create test data
    covariance = torch.tensor([
        [1.0, 1.2, 0.7], 
        [1.2, 1.0, -1.9], 
        [0.7, -1.9, 1.0]
    ])
    expected_return = torch.tensor([0.2, 0.05, 0.17])
    
    # Run optimization
    portfolio, gain = optimize_markowitz(
        covariance,
        expected_return,
        risk_coefficient=1,
        number_of_bits=3,
        agents=10,
        max_steps=5000,
        verbose=True
    )
    
    print("\nResults:")
    print(f"Portfolio weights: {portfolio}")
    print(f"Expected gain: {gain:.4f}")

Starting Discrete Simulated Bifurcation optimization...
Step 0/5000
Step 100/5000
Step 200/5000
Step 300/5000
Step 400/5000
Step 500/5000
Step 600/5000
Step 700/5000
Step 800/5000
Step 900/5000
Step 1000/5000
Step 1100/5000
Step 1200/5000
Step 1300/5000
Step 1400/5000
Step 1500/5000
Step 1600/5000
Step 1700/5000
Step 1800/5000
Step 1900/5000
Step 2000/5000
Step 2100/5000
Step 2200/5000
Step 2300/5000
Step 2400/5000
Step 2500/5000
Step 2600/5000
Step 2700/5000
Step 2800/5000
Step 2900/5000
Step 3000/5000
Step 3100/5000
Step 3200/5000
Step 3300/5000
Step 3400/5000
Step 3500/5000
Step 3600/5000
Step 3700/5000
Step 3800/5000
Step 3900/5000
Step 4000/5000
Step 4100/5000
Step 4200/5000
Step 4300/5000
Step 4400/5000
Step 4500/5000
Step 4600/5000
Step 4700/5000
Step 4800/5000
Step 4900/5000

Optimization complete!
Best portfolio: tensor([0., 7., 7.])
Expected gain: 45.6400

Results:
Portfolio weights: tensor([0., 7., 7.])
Expected gain: 45.6400


In [3]:

assert torch.equal(torch.tensor([0.0, 7.0, 7.0]), portfolio)
